<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-изучение-данных" data-toc-modified-id="Загрузка-и-изучение-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и изучение данных</a></span></li><li><span><a href="#Разбивка-данных-на-выборки" data-toc-modified-id="Разбивка-данных-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбивка данных на выборки</a></span></li><li><span><a href="#Исследование-модели" data-toc-modified-id="Исследование-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование модели</a></span><ul class="toc-item"><li><span><a href="#Исследуем-модель-дерева-решений" data-toc-modified-id="Исследуем-модель-дерева-решений-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Исследуем модель дерева решений</a></span></li><li><span><a href="#Исследуем-модель-случайного-леса" data-toc-modified-id="Исследуем-модель-случайного-леса-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Исследуем модель случайного леса</a></span></li><li><span><a href="#Исследуем-модель-логистической-регрессии" data-toc-modified-id="Исследуем-модель-логистической-регрессии-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Исследуем модель логистической регрессии</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверка-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-модели-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка модели на тестовой выборке</a></span></li><li><span><a href="#Проверка-модели-на-адекватность" data-toc-modified-id="Проверка-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка модели на адекватность</a></span><ul class="toc-item"><li><span><a href="#Результат-проверки-на-вменяемость" data-toc-modified-id="Результат-проверки-на-вменяемость-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Результат проверки на вменяемость</a></span></li></ul></li></ul></div>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не требуется.

Необходимо построить модель с максимально большим значением *accuracy*.  

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.  

Столбцы данных:  
- сalls — количество звонков,  
- minutes — суммарная длительность звонков в минутах,  
- messages — количество sms-сообщений,  
- mb_used — израсходованный интернет-трафик в Мб,  
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  

## Загрузка и изучение данных

Импортируем библиотеки

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm_notebook
import os

Откроем данные

In [2]:
pth1 = 'users_behavior.csv'
pth2 = 'https://code.s3.yandex.net/datasets/users_behavior.csv'
    
if os.path.exists(pth1):
    users_behavior = pd.read_csv(pth1)
elif os.path.exists(pth2):
    users_behavior = pd.read_csv(pth2)
else:
    print('Something is wrong')

Изучим данные

In [3]:
users_behavior.head() #покажем первые 5 строк данных

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
users_behavior.info() #покажем общую информацию о данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
users_behavior.describe() #покажем статистическую информацию о данных

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
users_behavior['is_ultra'].value_counts() # посмотрим количество элементов в каждом классе

0    2229
1     985
Name: is_ultra, dtype: int64

In [7]:
users_behavior['is_ultra'].mean() 
# посмотрим соотношение количеств элементов в каждом классе. Так как у нас классы 1 и 0, 
# то можно использовать среднеарифметическое значение

0.30647168637212197

## Разбивка данных на выборки

Создадим две таблицы из исходных данных - с параметрами и целевыми параметрами

In [8]:
features = users_behavior.drop(['is_ultra'], axis=1) #создадим таблицу с параметрами
target = users_behavior['is_ultra'] #создадим таблицу с целевыми параметрами

Разобьем данные на обучающую, валидационную и тестовую выборки

In [9]:
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345, 
    stratify=target)
#разобъем на обучающе-валидационную и тестовую выборки

In [10]:
print(features_train_valid.shape)
print(features_test.shape)
print(target_train_valid.shape)
print(target_test.shape)

(2571, 4)
(643, 4)
(2571,)
(643,)


Количества строк и столбцов соответствуют разбиению

In [11]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train_valid, target_train_valid, test_size=0.25, random_state=12345, 
    stratify=target_train_valid)
#разобъем на обучающую и валидационную выборки

In [12]:
print(features_train.shape)
print(features_valid.shape)
print(target_train.shape)
print(target_valid.shape)

(1928, 4)
(643, 4)
(1928,)
(643,)


Количества строк и столбцов соответствуют разбиению

In [13]:
target_train_valid.mean()

0.30649552703228317

In [14]:
target_test.mean()

0.30637636080870917

In [15]:
target_train.mean()

0.3065352697095436

In [16]:
target_valid.mean()

0.30637636080870917

Соотношение классов соответствует исходному набору данных

## Исследование модели

### Исследуем модель дерева решений

In [17]:
best_model_dtc = None
best_result_dtc = 0
best_max_depth_dtc = 0
best_min_samples_split_dtc = 0
best_min_samples_leaf_dtc = 0
for depth in range(1, 30):
    for min_samples in range(2,20):
        for leaf in range(1,20):
            model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=depth, 
                                               min_samples_split=min_samples,
                                              min_samples_leaf=leaf) 
            # обучим модель с заданными параметрами деревьев
            model_dtc.fit(features_train, target_train) # обучим модель на тренировочной выборке
            result_dtc = model_dtc.score(features_valid, target_valid) 
            # посчитаем качество модели на валидационной выборке
            if result_dtc > best_result_dtc:
                best_model_dtc = model_dtc # сохраним наилучшую модель
                best_result_dtc = result_dtc # сохраним наилучшее значение метрики accuracy на валидационных данных
                best_max_depth_dtc = depth # сохраним значение max_depth
                best_min_samples_split_dtc = min_samples # сохраним значение min_samples_split
                best_min_samples_leaf_dtc = leaf # сохраним значение min_samples_leaf

print("Accuracy наилучшей модели на валидационной выборке:", best_result_dtc, 
      "max_depth =", best_max_depth_dtc, 
      "best_min_samples_split =", best_min_samples_split_dtc, 
      "best_min_samples_leaf =", best_min_samples_leaf_dtc)

Accuracy наилучшей модели на валидационной выборке: 0.8195956454121306 max_depth = 7 best_min_samples_split = 12 best_min_samples_leaf = 4


Наиболее высокая точность предсказания с использованием модели дерева решений на валидационной выборке составила 0.819  
со следующими параметрами: max_depth = 7, min_samples_split = 12, min_samples_leaf = 4

### Исследуем модель случайного леса

In [18]:
%%time

best_model_rfc = None
best_result_rfc = 0
best_est_rfc = 0
for i in tqdm_notebook(range(10)):
    
    for est in range(1, 10):
        for depth in range(1, 10):
            for split in range(2,10):
                for leaf in range(1,10):
                    model_rfc = RandomForestClassifier(random_state=12345, n_estimators=est,\
                                                   max_depth=depth, \
                                                   min_samples_split=split, \
                                                   min_samples_leaf=leaf)
                    model_rfc.fit(features_train, target_train) # обучим модель на тренировочной выборке
                    result_rfc = model_rfc.score(features_valid, target_valid) 
                    # посчитаем качество модели на валидационной выборке
                    if result_rfc > best_result_rfc:
                        best_model_rfc = model_rfc # сохраним наилучшую модель
                        best_result_rfc = result_rfc 
                        # сохраним наилучшее значение метрики accuracy на валидационных данных
                        best_est_rfc = est # сохраним значение n_estimators
                        best_max_depth_rfc = depth # сохраним значение max_depth
                        best_min_samples_split_rfc = split # сохраним значение min_samples_split
                        best_min_samples_leaf_rfc = leaf # сохраним значение min_samples_leaf
    pass
print("Accuracy наилучшей модели на валидационной выборке:", best_result_rfc, "n_estimator =", best_est_rfc,\
     "max_depth =", best_max_depth_rfc, "best_min_samples_split =", best_min_samples_split_rfc, \
      "best_min_samples_leaf =", best_min_samples_leaf_rfc)

<timed exec>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/10 [00:00<?, ?it/s]

Accuracy наилучшей модели на валидационной выборке: 0.8351477449455676 n_estimator = 7 max_depth = 8 best_min_samples_split = 7 best_min_samples_leaf = 2
CPU times: user 17min 25s, sys: 11.5 s, total: 17min 36s
Wall time: 17min 48s


Наиболее высокая точность предсказания с использованием модели случайного на валидационной выборке составила 0,835  
со следующими параметрами: n_estimator = 7, max_depth = 8, min_samples_split = 7, min_samples_leaf = 2

In [19]:
%%time
clf = RandomForestClassifier(random_state=12345)
parametrs = {'n_estimators': range(1, 10),
              'max_depth': range(1,10),
              'min_samples_split': range(2,10)}
grid = GridSearchCV(clf, parametrs)
grid.fit(features_train, target_train)
print(grid.best_params_)
print(grid.best_score_)

{'max_depth': 9, 'min_samples_split': 8, 'n_estimators': 9}
0.8070587443644438
CPU times: user 1min 7s, sys: 1.34 s, total: 1min 9s
Wall time: 1min 12s


In [20]:
model_rfc = RandomForestClassifier(random_state=12345, **grid.best_params_)
model_rfc.fit(features_train, target_train)
result_rfc = model_rfc.score(features_valid, target_valid)
print(result_rfc)

0.8242612752721618


### Исследуем модель логистической регрессии

In [21]:
%%time
clf_lr = LogisticRegression(random_state=12345)
grid_values = {'penalty': ['l1','l2'],
               'C': [0.001,0.01,0.1,1,10,100,1000],
              'solver': ['newton-cg', 'lbfgs', 'saga']}
grid_lr = GridSearchCV(clf_lr, grid_values)
grid_lr.fit(features_train, target_train)
print(grid_lr.best_params_)
print(grid_lr.best_score_)

/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/User

/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: Line

/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:437: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The lin

/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: Line

/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/lines

/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/lines

/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.

/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.

{'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
0.7453334230536303
CPU times: user 16.4 s, sys: 393 ms, total: 16.8 s
Wall time: 9.7 s


/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


In [22]:
model_lr_2 = LogisticRegression(random_state=12345, **grid_lr.best_params_)
model_lr_2.fit(features_train, target_train)
model_lr_2.score(features_valid, target_valid)

/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:478: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/antonina/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)


0.7558320373250389

Используя подбор параметров удалось добиться точности 0.7558

In [23]:
model_lr = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)

In [24]:
model_lr.fit(features_train, target_train)

LogisticRegression(max_iter=1000, random_state=12345)

In [25]:
model_lr.score(features_valid, target_valid)

0.7558320373250389

Точность предсказания с использованием метода логистической регрессии на валидационной выборке составила 0,755

### Вывод
В результате исследования трех моделей классификации были получены следующие результаты:
- Модель случайного леса с гиперпараметрами n_estimator = 7, max_depth = 8, min_samples_split = 7, min_samples_leaf = 2 показала наибольшее значение точности предсказания на валидационной выборке - 0,835
- Модель дерева решений с гиперпараметрами max_depth = 10, min_samples_split = 10, min_samples_leaf = 8 показала второе значение точности предсказания на валидационной выборке - 0,824
- Модель логистической регрессии показала значение точности предсказания на валидационной выборке - 0,755.

Результаты совпрадают с ожиданием: так как случайный лес является ансамблем дерева решений - выдает усредненное значение нескольких деревьев решений, то он требует больших затрат на вычисление, но и его результат является наиболее точным

## Проверка модели на тестовой выборке

Обучим лучшую модель (случайный лес) на обучающе-валидационной выборке

In [26]:
result = model_rfc.score(features_test, target_test)
print(result)

0.807153965785381


**Результат проверки на тестовой выборке**  
Модель случайного леса с получеными параметрами дала точность 0.8071

## Проверка модели на адекватность

Проверим модель на вменяемость, используя DummyClassifier, strategy - наиболее часто встречающиеся

In [27]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train_valid, target_train_valid)
dummy_clf.predict(features_test)
dummy_clf.score(features_test, target_test)

0.6936236391912908

### Результат проверки на вменяемость
Точность предсказания нашей модели (0.78849) значительно больше модели, используещую для предсказания наболее часто встречающееся значение